# Some Exploratory Data Analysis

The goal of this notebook is to do some EDA and plotting of the data to get a better feel for it. In particular we will look at time series plots of certain features of interest.

## **A. Imports**

In [ ]:
from google.cloud import bigquery
from google.cloud.bigquery import job

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

## **B. BigQuery**

**What is this query doing?**

We look through the dataset and find every "failure" instance. We also randomly sample a roughly equal number of "non failure" instances, where we define a "non failure" in this scenario to mean not failing in the next 30 days (fail_this_month=0). Now for each of these "failure" and "non failure" instances, we extract a 30 day history. 

**Why?**

Our goal is visualize time series of different SMART features. We want to accomplish two things:
 + What do the time series generally look like (are there interesting trends/patterns)?
 + How do the time series and/or average values for different features vary between "failure" and "non failure" instances?

In [ ]:
FULL_NAME_OF_TABLE = # Set this as `{project}.{dataset}.{table}`

QUERY = """
WITH failures AS (
SELECT 
  serial_number,
  date as last_day
FROM 
  {0}
WHERE failure=1
AND date < '2019-01-01'),

non_failures AS (
SELECT
  serial_number,
  date as last_day
FROM 
  {0}
WHERE fail_this_month=0
AND date < '2019-01-01'
AND MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(STRUCT(date,serial_number)))), 
    (SELECT COUNT(*) FROM {0})) < 
    (SELECT COUNT(*) FROM failures))

SELECT 
  *,
  30 - DATE_DIFF(last_day, date, day) as day_in_window
FROM (
SELECT
  * 
FROM 
  non_failures 
JOIN {0}
USING(serial_number)
WHERE DATE_DIFF(last_day, date, day) < 30 AND DATE_DIFF(last_day, date, day) >= 0 

UNION ALL

SELECT 
  *
FROM 
  failures
JOIN {0}
USING(serial_number)
WHERE DATE_DIFF(last_day, date, day) < 30)
ORDER BY serial_number, date
""".format(FULL_NAME_OF_TABLE)

client = bigquery.Client()
query_job = client.query(QUERY)

frame = query_job.result().to_dataframe()

In [ ]:
display(frame)

---
## **C. Visualizing Time Series**

Now we have about 186,266 rows, corresponding to about 6000 different 30 day windows. Half of these windows (~3500) are meant to illustrate "normal operating behavior" while in the other half of these 30 day windows the drive will fail on the final day.

The first thing we can look at is an average of these 30 day time series for particular features. From [a page](https://www.backblaze.com/blog/what-smart-stats-indicate-hard-drive-failures/) on the BackBlaze website we know that there are 5 features that the company looks at in order to predict drive failure: 5, 187, 188, 197, 198. We can start by looking at these five features (we ignore smart_188_raw for now since it is null for over 50% of the rows in the table).

In [ ]:
def make_labels(features):
    """Make labels for x-axis"""
    labels = [feat.split('_')[0].upper() + ' ' + feat.split('_')[1] 
              for feat in features]
    return labels

# From domain knowledge, we know that 5 features tend to be predictive of failure
FEATURES_OF_INTEREST = ['smart_5_raw', 'smart_187_raw', 'smart_188_raw', 
                        'smart_197_raw', 'smart_198_raw'] 
LABELS = make_labels(FEATURES_OF_INTEREST)

def plot_time_series(frame, sample=False, features=FEATURES_OF_INTEREST, labels=LABELS, save=False):
    """
    Convenience function to plot some time series for some different features
    
    Parameters
    ----------
    frame : DataFrame
        A pandas DataFrame containing 30 day windows for healthy and infected drives
        
    sample : Bool
        If true plot sample time series for single hard drives. Otherwise plot averages
        for "healthy" and "fail" groups.
        
    features : List
        List of strings indicating which features we'd like to plot
        
    labels : List
        List of strings to be used as axis labels
    """
    fig, axes = plt.subplots(len(features), 1, figsize=(10, 10))
    
    # Generate a random sample
    if sample:
        # Get all of the serial numbers that did and did not fail
        no_fail_serial_numbers = frame[frame['fail_this_month']==0]['serial_number'].unique()
        fail_serial_numbers = frame[frame['fail_this_month']==1]['serial_number'].unique()
        # Get a random sample from each 
        no_fail_sample = frame[frame['serial_number'] == np.random.choice(no_fail_serial_numbers, 1)[0]]
        fail_sample = frame[frame['serial_number'] == np.random.choice(fail_serial_numbers, 1)[0]]
        
        title = "Sample Readings for Failed and Non-failed Drives for {} SMART Features".format(len(features))
    else:
        title = "Average Readings During 30-day Window for {} SMART Features".format(len(features))  
        
    for ax, feat, label in zip(axes, features, labels):
        if sample:
            sns.lineplot(no_fail_sample['day_in_window'], no_fail_sample[feat], ax=ax)
            sns.lineplot(fail_sample['day_in_window'], fail_sample[feat], ax=ax)
        else:
            sns.lineplot(x='day_in_window', y=feat, hue='fail_this_month', data=frame, ax=ax, legend=False)
        
        ax.set_xlabel('')
        ax.set_ylabel(label)
        
    fig.legend(['No Fail', 'Fail'], loc='upper left', bbox_to_anchor = (0.1,-0.04,1,1))
    fig.suptitle(title, va='bottom')
    axes[-1].set_xlabel('Day in Window')  
    plt.tight_layout()
    
    if save:
        if sample:
            plt.savefig('sample_readings.png')
        else:
            plt.savefig('avg_readings.png')

In [ ]:
plot_time_series(frame)

**What does this plot tell us?**

There's a lot of interesting conclusions that can be drawn from this plot: 
 + there is a noticeable difference between the average readings for these 5 features for drives that did or did not fail. 
 + the readings for SMART 197 and SMART 198 look nearly identical, this agrees with reporting by BackBlaze.
 + Non failed drives have higher average reading for SMART 188. This *does not* agree with reporting by BackBlaze—these state: from the linked page "when the RAW value for one of these five attributes is greater than zero, we have a reason to investigate". Plot would seem to indicate the opposite for SMART 188

### Samples

Time series look uninteresting for the most part (but this could be due to aggregation, we're only looking at averages). Lets look at individual samples to see if the time series still look uninteresting. This could help inform our modelling approach. 

In [ ]:
plot_time_series(frame, sample=True)

**NOTE**: Rerun the two cells above to see different samples

### Alternative Features

What happens if we do the same plots for some other features? Do we still see a noticeable difference between the two groups?

In [ ]:
# From domain knowledge, we know that these 5 features tend to be predictive of failure
new_features = ['smart_12_raw', 'smart_189_raw', 'smart_190_raw', 'smart_193_raw', \
                        'smart_199_raw', 'smart_240_raw', 'smart_242_raw']
new_labels = make_labels(new_features)

In [ ]:
plot_time_series(frame, features=new_features, labels=new_labels)